In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import scipy.stats
import pandas
import copy as cp

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

In [2]:
Product = pd.read_csv('data/Pruduct.csv')
Product = Product[Product['HITS_SEQ'] != 1]
Product.reset_index(inplace=True,drop=True)
Master = pd.read_csv('data/Master.csv')
Session = pd.read_csv("data/Session.csv")
Custom = pd.read_csv("data/Custom.csv")

In [3]:
Item = pd.merge(Product,Master,on=['PD_C'],how='left')
Item = Item[['CLNT_ID','SESS_ID','CLAC1_NM','CLAC3_NM']]
Item = Item.drop_duplicates()

In [4]:
# 한 세션당2개 품목 이상인 거래
item_size = Item.groupby(['CLNT_ID','SESS_ID'])['CLAC3_NM'].nunique().reset_index()
item_size_2 = item_size[item_size['CLAC3_NM']>1]
Item = pd.merge(Item,item_size_2,on=['CLNT_ID',"SESS_ID"],how='inner')
Item.rename(columns={'CLAC3_NM_x':'CLAC3_NM','CLAC3_NM_y':'item_size'},inplace=True)

In [5]:
# 세션내 필요한 컬럼만 추출
Session = Session[['CLNT_ID','SESS_ID','SESS_SEQ']]

In [6]:
# Cold start 문제
# 구매 이력이 하나밖에 없는 User
# 구매 이력이 하나밖에 없으면 train시킬 건수는 있지만 test시킬 건수가 없다
# 따로 이런 유저들은 어떻게 구매가 이루어졌는지 확인

item_clntid = Item.groupby(['CLNT_ID'])['SESS_ID'].nunique().reset_index()
item_clntid_1 = item_clntid[item_clntid['SESS_ID']==1]
Item_1 = pd.merge(Item,item_clntid_1,on=['CLNT_ID'],how='inner')
Item_1.rename(columns={'SESS_ID_y':'sess_uni_c','SESS_ID_x':'SESS_ID'},inplace=True)

# 세션내에 구매가 된 상품들을 모아줌
Item_1['CLAC3_NM'] = Item_1['CLAC3_NM']+","
Item_1 = Item_1.groupby(['CLNT_ID',"SESS_ID"])['CLAC3_NM'].sum().reset_index()
Item_1['CLAC3_NM'] = Item_1['CLAC3_NM'].apply(lambda x: x.rstrip(','))

# CLNT_AGE, CLNT_GENDER 에 있는 nan값 제거
Item_1 = pd.merge(Item_1,Session,on=['CLNT_ID','SESS_ID'],how='left')
Item_1 = pd.merge(Item_1,Custom,on=['CLNT_ID'],how='left')
Item_1 = Item_1[Item_1['CLNT_AGE'].notnull()]

In [7]:
Item_1.head()

,CLNT_ID,SESS_ID,CLAC3_NM,SESS_SEQ,CLNT_GENDER,CLNT_AGE
0,32,1136737,"남성티셔츠,남성캐주얼재킷",99,M,40.0
2,58,4328876,"남성티셔츠,남성캐주얼바지,남성스포츠티셔츠",8,F,30.0
3,59,425903,"기타조리도구,식기건조대/수저통,국자/뒤지개/주걱",126,F,40.0
4,68,4610298,"음료용컵,주류잔",30,F,30.0
5,71,4825972,"여성티셔츠/탑,남성일반스포츠바지",19,F,40.0


In [8]:
# 10번 이상 거래내역이 있는 고객
item_clntid = Item.groupby(['CLNT_ID'])['SESS_ID'].nunique().reset_index()
item_clntid_10 = item_clntid[item_clntid['SESS_ID']>=10]
Item = pd.merge(Item,item_clntid_10,on=['CLNT_ID'],how='inner')
Item.rename(columns={'SESS_ID_y':'sess_uni_c','SESS_ID_x':'SESS_ID'},inplace=True)

# 세션내에 구매가 된 상품들을 모아줌
Item['CLAC3_NM'] = Item['CLAC3_NM']+","
Item = Item.groupby(['CLNT_ID',"SESS_ID"])['CLAC3_NM'].sum().reset_index()
Item['CLAC3_NM'] = Item['CLAC3_NM'].apply(lambda x: x.rstrip(','))

# CLNT_AGE, CLNT_GENDER 에 있는 nan값 제거
Item = pd.merge(Item,Session,on=['CLNT_ID','SESS_ID'],how='left')
Item = pd.merge(Item,Custom,on=['CLNT_ID'],how='left')
Item = Item[Item['CLNT_AGE'].notnull()]

In [9]:
def top_2_n(df, n=3):

    return df.sort_values(by=["SESS_SEQ"], ascending=False)[1:n]


def top_1(df):

    return df.sort_values(by=["SESS_SEQ"], ascending=False)[:1]

In [10]:
# 10개 이상의 세션을 갖는 CLNT_ID 중에서 가장 최근 구매가 이루어진 세션 TOP 2 - 10
Item_train = Item.groupby(['CLNT_ID']).apply(top_2_n,n=10)
Item_train.reset_index(inplace=True,drop=True)

In [11]:
Item_train.head(10)

,CLNT_ID,SESS_ID,CLAC3_NM,SESS_SEQ,CLNT_GENDER,CLNT_AGE
0,451,4327763,"여성샌들,섬유유연제/향기지속제",615,F,40.0
1,451,4404129,"여성스니커즈,남성슬립온",608,F,40.0
2,451,4464232,"여성티셔츠/탑,여성스커트",604,F,40.0
3,451,5050230,"립글로즈/틴트,페이셜클렌저,스킨/토너,바디워시,헤어에센스,여성샌들,여성바지,여성원피...",578,F,40.0
4,451,5573240,"바디워시,트리트먼트/팩,페이셜클렌저,액상세탁세제",561,F,40.0
5,451,7082059,"트리트먼트/팩,바디워시,헤어케어선물세트",522,F,40.0
6,451,7582904,"여성토트백,여성스커트,여성티셔츠/탑,섬유유연제/향기지속제",498,F,40.0
7,451,7860136,"선크림류,헤어케어선물세트",493,F,40.0
8,451,9532885,"여성티셔츠/탑,여성점퍼",460,F,40.0
9,918,3122439,"애견간식,애견목욕/위생용품",191,M,40.0


In [12]:
# 가장 최근 구매가 이루어진 세션 TOP 1
Item_test = Item.groupby(['CLNT_ID']).apply(top_1)
Item_test.reset_index(inplace=True,drop=True)

In [13]:
Item_test.head(10)

,CLNT_ID,SESS_ID,CLAC3_NM,SESS_SEQ,CLNT_GENDER,CLNT_AGE
0,451,2074820,"샴푸,립글로즈/틴트,칫솔,트리트먼트/팩,아이브로우,치약,액상세탁세제,섬유유연제/향기...",689,F,40.0
1,918,2463335,"여성티셔츠/탑,여성원피스",198,M,40.0
2,1554,2187612,"유아용욕조,분말세탁세제",1285,F,40.0
3,4619,2066760,"여성재킷,남성런닝/트레이닝화,여성바지,여성티셔츠/탑",1329,F,30.0
4,7263,2838222,"냄비,프라이팬",855,F,30.0
5,7618,2298502,"린스/컨디셔너,샴푸",802,F,40.0
6,7708,2342522,"아쿠아슈즈,선크림류",1461,F,50.0
7,7842,2305199,"여성남방셔츠,여성재킷",1098,F,50.0
8,8351,1424618,"여성스커트,여성재킷",1481,F,50.0
9,10818,2328672,"남성캐주얼바지,여성스커트,여성원피스,냅킨",700,F,60.0


In [14]:
print(Item_train.shape)
print(Item_test.shape)

(43101, 6)
(4789, 6)


In [15]:
Item_train.to_csv("./doc2vec/Item_train.csv",index=False)
Item_test.to_csv("./doc2vec/Item_test.csv",index=False)

In [16]:
# Train과 Test데이터셋을 구분하기위해 만든 상품리스트를 다시 나눠줌
Item_train['CLAC3_NM'] = Item_train['CLAC3_NM'].apply(lambda x: x.split(','))
a = Item_train['CLAC3_NM'].apply(lambda x: pd.Series(x))
a = a.stack().reset_index(level=1,drop=True).to_frame('CLAC3_NM')
Item_train = Item_train.merge(a,left_index=True,right_index=True,how='left').reset_index(drop=True)
Item_train.rename(columns={'CLAC3_NM_y':'CLAC3_NM'},inplace=True)

In [17]:
Item_test['CLAC3_NM'] = Item_test['CLAC3_NM'].apply(lambda x: x.split(','))
a = Item_test['CLAC3_NM'].apply(lambda x: pd.Series(x))
a = a.stack().reset_index(level=1,drop=True).to_frame('CLAC3_NM')
Item_test = Item_test.merge(a,left_index=True,right_index=True,how='left').reset_index(drop=True)
Item_test.rename(columns={'CLAC3_NM_y':'CLAC3_NM'},inplace=True)

In [18]:
print(Item_train.shape)
print(Item_test.shape)
print(Item_1.shape)

(120662, 7)
(13122, 7)
(180199, 6)


In [19]:
# 한번만 구매된 CLNT_ID들 재 분해
Item_1['CLAC3_NM'] = Item_1['CLAC3_NM'].apply(lambda x: x.split(','))
a = Item_1['CLAC3_NM'].apply(lambda x: pd.Series(x))
a = a.stack().reset_index(level=1,drop=True).to_frame('CLAC3_NM')
Item_1 = Item_1.merge(a,left_index=True,right_index=True,how='left').reset_index(drop=True)
Item_1.rename(columns={'CLAC3_NM_y':'CLAC3_NM'},inplace=True)

In [20]:
Item_train.to_csv("./doc2vec/Item_train_flat.csv",index=False)
Item_test.to_csv("./doc2vec/Item_test_flat.csv",index=False)
Item_1.to_csv("./doc2vec/Item_test_flat.csv",index=False)

- User Features data

In [21]:
train_user_features = Item_train[['CLNT_ID','SESS_ID','CLNT_GENDER','CLNT_AGE'
#                                   ,'DVC_CTG_NM_y','ZON_NM_y'
                                 ]].drop_duplicates()
train_user_features.drop(['CLNT_ID','SESS_ID'],inplace=True,axis=1)
test_user_features = Item_test[['CLNT_ID','SESS_ID','CLNT_GENDER','CLNT_AGE'
#                                 ,'DVC_CTG_NM_y','ZON_NM_y'
                               ]].drop_duplicates()
test_user_features.drop(['CLNT_ID','SESS_ID'],inplace=True,axis=1)

In [22]:
user_features_1 = Item_1[['CLNT_ID','SESS_ID','CLNT_GENDER','CLNT_AGE']].drop_duplicates()
user_features_1.drop(['CLNT_ID','SESS_ID'],inplace=True,axis=1)

In [23]:
print(train_user_features.shape)
print(test_user_features.shape)
print(user_features_1.shape)

(43101, 2)
(4789, 2)
(180199, 2)


In [24]:
train_user_features.to_csv("./doc2vec/train_user_features.csv",index=False)
test_user_features.to_csv("./doc2vec/test_user_features.csv",index=False)
user_features_1.to_csv("./doc2vec/user_features_1.csv",index=False)

- label data

In [25]:
# 한번만 구매한 label
data_1 = Item_1[['CLAC3_NM','SESS_ID','CLNT_ID']]

data_1_label = pd.get_dummies(data_1['CLAC3_NM'])
data_1_label['CLNT_ID'] = data_1['CLNT_ID']
data_1_label['SESS_ID'] = data_1['SESS_ID']

data_1_label.reset_index(drop=True,inplace=True)

In [26]:
data_1_label = data_1_label.groupby(['CLNT_ID','SESS_ID']).sum().reset_index()
data_1_label.drop(['CLNT_ID',"SESS_ID"],axis=1,inplace=True)

In [27]:
train_data = Item_train[['CLAC3_NM','SESS_ID','CLNT_ID']]
test_data = Item_test[['CLAC3_NM','SESS_ID','CLNT_ID']]

data = pd.concat([train_data,test_data])

In [28]:
data_label = pd.get_dummies(data['CLAC3_NM'])
data_label['CLNT_ID'] = data['CLNT_ID']
data_label['SESS_ID'] = data['SESS_ID']

data_label.reset_index(drop=True,inplace=True)

In [29]:
train_label = data_label[:train_data.shape[0]]
test_label = data_label[train_data.shape[0]:]

In [30]:
train_label = train_label.groupby(['CLNT_ID','SESS_ID']).sum().reset_index()
test_label = test_label.groupby(['CLNT_ID','SESS_ID']).sum().reset_index()

train_label.drop(['CLNT_ID',"SESS_ID"],axis=1,inplace=True)
test_label.drop(['CLNT_ID',"SESS_ID"],axis=1,inplace=True)

In [31]:
train_label.to_csv("./doc2vec/train_label2.csv",index=False)
test_label.to_csv("./doc2vec/test_label2.csv",index=False)

In [32]:
data_1_label.to_csv("./doc2vec/data_1_label.csv",index=False)

- Segment data

In [33]:
Item = pd.concat([Item_train,Item_test]).reset_index(drop=True)
Master = Master[["CLAC1_NM",'CLAC3_NM']].drop_duplicates()

Item = pd.merge(Item,Master,on=['CLAC3_NM'],how='left')

In [34]:
Segment = pd.get_dummies(Item[['CLAC1_NM','CLAC3_NM']].set_index("CLAC3_NM")).reset_index(drop=True)

Segment['CLNT_ID'] = Item['CLNT_ID']
Segment['SESS_ID'] = Item['SESS_ID']

In [35]:
# 한번만 구매한 Segment
Item_1 = pd.merge(Item_1,Master,on=['CLAC3_NM'],how='left')

Segment_1 = pd.get_dummies(Item_1[['CLAC1_NM','CLAC3_NM']].set_index("CLAC3_NM")).reset_index(drop=True)
Segment_1['CLNT_ID'] = Item_1['CLNT_ID']
Segment_1['SESS_ID'] = Item_1['SESS_ID']

Segment_1 = Segment_1.groupby(['CLNT_ID','SESS_ID']).sum().reset_index(drop=True)

In [36]:
train_Segment = Segment[:Item_train.shape[0]]
test_Segment = Segment[Item_train.shape[0]:]

In [37]:
train_Segment = train_Segment.groupby(['CLNT_ID','SESS_ID']).sum().reset_index(drop=True)
test_Segment = test_Segment.groupby(['CLNT_ID','SESS_ID']).sum().reset_index(drop=True)

In [38]:
print(train_Segment.shape)
print(test_Segment.shape)

(43101, 37)
(4789, 37)


In [39]:
train_Segment.to_csv("./doc2vec/train_Segment.csv",index=False)
test_Segment.to_csv("./doc2vec/test_Segment.csv",index=False)

In [40]:
Segment_1.to_csv("./doc2vec/Segment_1.csv",index=False)

In [41]:
print(train_data.shape)
print(train_Segment.shape)
print(train_label.shape)
print(train_user_features.shape)

(120662, 3)
(43101, 37)
(43101, 831)
(43101, 2)


In [42]:
print(test_data.shape)
print(test_label.shape)
print(test_Segment.shape)
print(test_user_features.shape)

(13122, 3)
(4789, 831)
(4789, 37)
(4789, 2)
